## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Qostanay,KZ,53.1667,63.5833,14.29,73,0,8.95,clear sky
1,1,Camacha,PT,33.0833,-16.3333,63.54,82,40,26.46,scattered clouds
2,2,Rikitea,PF,-23.1203,-134.9692,79.57,67,100,13.73,overcast clouds
3,3,Tomatlan,MX,19.9333,-105.2500,68.90,83,87,2.51,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,79.09,75,30,21.05,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 105


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,79.57,67,100,13.73,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,79.09,75,30,21.05,scattered clouds
9,9,Boa Vista,BR,2.8197,-60.6733,84.18,61,75,5.75,broken clouds
12,12,Porto Novo,BJ,6.4965,2.6036,82.22,90,0,3.00,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,78.96,89,100,16.11,light rain
18,18,Carnarvon,AU,-24.8667,113.6333,82.47,69,75,14.97,broken clouds
19,19,Mount Gambier,AU,-37.8333,140.7667,85.91,25,40,13.80,scattered clouds
20,20,Edd,ER,13.9297,41.6925,75.83,76,99,15.39,overcast clouds
27,27,Avarua,CK,-21.2078,-159.7750,82.45,78,20,16.11,few clouds
41,41,Souillac,MU,-20.5167,57.5167,79.36,94,40,10.36,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,79.57,67,100,13.73,overcast clouds
4,4,Atuona,PF,-9.8000,-139.0333,79.09,75,30,21.05,scattered clouds
9,9,Boa Vista,BR,2.8197,-60.6733,84.18,61,75,5.75,broken clouds
12,12,Porto Novo,BJ,6.4965,2.6036,82.22,90,0,3.00,clear sky
16,16,Vaini,TO,-21.2000,-175.2000,78.96,89,100,16.11,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.57,overcast clouds,-23.1203,-134.9692,
4,Atuona,PF,79.09,scattered clouds,-9.8000,-139.0333,
9,Boa Vista,BR,84.18,broken clouds,2.8197,-60.6733,
12,Porto Novo,BJ,82.22,clear sky,6.4965,2.6036,
16,Vaini,TO,78.96,light rain,-21.2000,-175.2000,
18,Carnarvon,AU,82.47,broken clouds,-24.8667,113.6333,
19,Mount Gambier,AU,85.91,scattered clouds,-37.8333,140.7667,
20,Edd,ER,75.83,overcast clouds,13.9297,41.6925,
27,Avarua,CK,82.45,few clouds,-21.2078,-159.7750,
41,Souillac,MU,79.36,scattered clouds,-20.5167,57.5167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        hotel_df['Hotel Name'].replace('', np.nan, inplace=True)      

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.57,overcast clouds,-23.1203,-134.9692,People ThankYou
4,Atuona,PF,79.09,scattered clouds,-9.8000,-139.0333,Villa Enata
9,Boa Vista,BR,84.18,broken clouds,2.8197,-60.6733,Aipana Plaza Hotel
12,Porto Novo,BJ,82.22,clear sky,6.4965,2.6036,Tour Eiffel Hotel Benin
16,Vaini,TO,78.96,light rain,-21.2000,-175.2000,Keleti Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))